In [56]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # dirty hack, but ok for research
from utils import img_collection as icoll
import cv2
import numpy as np
from sklearn.externals import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler

In [11]:
trainPool = []

for i, r in enumerate(icoll.getPoolUrlsIterator('581e06de0310e9180b1f7a62')):
    if r['downloadable']:
        trainPool.append(r)
    if i>10:
        break


In [10]:
# Create feature extraction and keypoint detector objects
fea_det = cv2.FeatureDetector_create("SIFT")
des_ext = cv2.DescriptorExtractor_create("SIFT")

In [90]:
# List where all the descriptors are stored
des_list = []

for imageRow in trainPool:
    im = cv2.imread(imageRow['path'])
    if im is None:
        continue
    kpts = fea_det.detect(im)
    kpts, des = des_ext.compute(im, kpts)
    des_list.append(des)

In [91]:
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
for descriptor in des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))  

In [92]:
# Perform k-means clustering
k = 100
voc, variance = kmeans(descriptors, k, 1) 

In [94]:
voc.shape

(100, 128)

In [95]:
words, distance = vq(des_list[0],voc)

In [96]:
words

array([99, 36, 26, 69, 74, 46, 54, 17, 32, 74, 64, 74, 79, 29, 79,  9, 63,
       20, 65, 70, 25, 12, 55, 63, 19, 61, 87, 16, 77, 46, 33, 58, 79, 75,
        9, 46, 90,  6, 61, 52,  3, 82, 83, 82, 20,  3, 38, 50, 91, 90, 88,
       98, 87, 19, 27,  5, 23, 58, 87, 87, 95, 95, 58, 24, 64, 67,  5, 49,
        5, 90, 98, 98, 16, 52, 96, 82,  3, 81, 76, 16, 53, 25, 25, 49, 84,
       20, 99, 32, 61, 33, 32,  7, 72, 29, 95, 55, 71, 92, 75, 97, 50, 12,
       25, 27, 56, 72, 98, 33, 72, 44, 19, 44, 54, 90, 95, 95, 33, 25, 56,
       36, 99, 33, 83, 43, 37, 69, 46, 91, 39,  7, 79, 37, 56, 46, 74, 12,
       39,  6, 37, 83, 57, 80,  3, 32, 46,  8, 83, 74, 29, 23, 96, 83, 55,
       37, 45,  3, 95, 37, 27, 64,  3, 24, 48, 32, 98, 20, 12, 50, 16, 69,
       86, 98, 87, 87, 61,  3, 98, 87, 43, 22, 29, 86,  9, 62, 47,  4, 22,
       90, 84, 19, 46,  2, 59, 29, 90, 82, 89,  6, 33, 13, 27, 37, 46, 50,
       75, 64, 49, 27, 49, 20, 54, 52, 46, 36, 34, 51, 96,  2, 54, 62, 56,
       65, 61,  6, 30, 79

In [97]:
imFeatures = np.zeros((len(des_list), k), "float32")
for i in xrange(len(des_list)):
    words, distance = vq(des_list[i],voc)
    for w in words:
        imFeatures[i][w] += 1

# Scaling the words
stdSlr = StandardScaler().fit(imFeatures)
joblib.dump((stdSlr, k, voc), "SIFT_bow_k100.pkl", compress=3)
#imFeatures = stdSlr.transform(imFeatures)



['SIFT_bow_k100.pkl']

In [100]:
stdSlr.transform(imFeatures).std()

1.0

In [101]:
stdSlr_f, k_f, voc_f = joblib.load("SIFT_bow_k100.pkl")

In [102]:
trainPool_f = []
for i, r in enumerate(icoll.getPoolUrlsIterator('581e06de0310e9180b1f7a62')):
    if i < 10:
        continue
    if r['downloadable']:
        trainPool_f.append(r)
    if i>20:
        break

In [110]:
for imageRow in trainPool_f:
    im = cv2.imread(imageRow['path'])
    if im is None:
        continue
    kpts = fea_det.detect(im)
    kpts, des = des_ext.compute(im, kpts)
    imFeatures = np.zeros((1, k), "float32")
    words, distance = vq(des, voc_f)
    for w in words:
        imFeatures[0][w] += 1
    imFeatures = stdSlr_f.transform(imFeatures)
    print imFeatures
    #des_list.append(des)   

[[  3.78398013   1.8584255    0.4735027    1.8219105    1.38695228
    4.81818199   1.04194677   1.33891118   0.60903192   8.74507523
    0.88354117   0.99450541  12.84634495   0.36438999   1.53473592
   -0.4107919    0.75013393  -0.18428855  -0.55131775   2.70489478
    2.22715569   3.54521608  -0.36164927   7.01322746   0.25        -1.03563726
    1.78885436  -0.6123724    0.9799118    2.56986475   0.38263932
    0.4186379    4.52409315   0.20686899  -0.3273614    4.71952438
    0.31446508   3.8144691    1.80578792   3.63759875   1.82689488
    1.99323189   0.3499271    0.25997347  -1.24656177  -0.86179805
    0.73026776   0.4747127    1.76190472   0.83683223   0.32051325
    0.06406222  -0.55391169   0.7813018   -1.21948171   6.74631643
   -0.7970534    5.30830336   1.89985311   2.52347469   1.53392982
    1.8641907    1.59719133   1.44625545  -0.12666011   0.74039161
    0.20961082   2.62379432   1.08253181  -0.42857143   0.65991199
   -0.34554741  -0.58843899  -0.12194213   4.3561

In [104]:
des.shape

(1925, 128)